In [1]:
# notebook to illustrate document similarity clusters

In [2]:
# following only used for development, reloads the modules with any code changes
%load_ext autoreload
%autoreload 2

# inline matplotlib charts
%matplotlib inline

In [3]:
# import our text mining toolkit
import text_mining_toolkit as tmt

In [4]:
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/recipes/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(directory_of_files="data_sets/mystery_corpus_01/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/iraq_inquiry/txt/", text_filename_pattern="the-report*.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/clinton_emails/txt/", text_filename_pattern="C0*.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/shakespeare_macbeth/txt/", text_filename_pattern="macbeth_act_0?_scene_0?.txt")
cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/mixed/txt/", text_filename_pattern="*.txt")

content_directory =  data_sets/mixed/txt/
text_filename_pattern =  *.txt
self.documents populated =  52


In [5]:
# first clear index
tmt.index_wordcount.delete_index(cr.content_directory)

# for all documents in corpus
for document_name in cr.get_documents():
    print("processing ", document_name)

    # get document text
    document_text = cr.get_text_by_document(document_name)

    # simplify whitespace (remove newlines)
    b = tmt.text_processing.simplify_whitespace(document_text)

    # only keep alphanumeric characters, removes punctuation
    c = tmt.text_processing.keep_only_alphanumeric(b)

    # make lowercase
    d = tmt.text_processing.to_lowercase(c)

    # split into words list
    dl = tmt.text_processing.split_text_into_words(d)
    
    # build n-grams
    #gl = tmt.word_processing.build_ngrams_from_words(dl,2)

    # remove stop words
    #el = tmt.word_processing.remove_stop_words(dl, "./stopwords/minimal-stop.txt")
    
    # update index
    tmt.index_wordcount.create_wordcount_index_for_document(cr.content_directory, document_name, dl)
    pass



removed wordcount index file:  data_sets/mixed/txt/index_wordcount.hdf5
processing  01.txt
processing  02.txt
processing  03.txt
processing  04.txt
processing  05.txt
processing  06.txt
processing  07.txt
processing  08.txt
processing  09.txt
processing  10.txt
processing  11.txt
processing  12.txt
processing  13.txt
processing  C05739545.txt
processing  C05739546.txt
processing  C05739547.txt
processing  C05739550.txt
processing  C05739554.txt
processing  C05739559.txt
processing  C05739560.txt
processing  C05739561.txt
processing  C05739562.txt
processing  C05739563.txt
processing  C05739564.txt
processing  C05739565.txt
processing  C05739567.txt
processing  macbeth_act_01_scene_01.txt
processing  macbeth_act_01_scene_02.txt
processing  macbeth_act_01_scene_03.txt
processing  macbeth_act_01_scene_04.txt
processing  macbeth_act_01_scene_05.txt
processing  macbeth_act_01_scene_06.txt
processing  macbeth_act_01_scene_07.txt
processing  macbeth_act_02_scene_01.txt
processing  macbeth_act

In [6]:
# merge document indices into a corpus index
tmt.index_wordcount.merge_wordcount_indices_for_corpus(cr.content_directory)

saving corpus word count index ...  data_sets/mixed/txt/index_wordcount.hdf5


In [7]:
# delete and build relevance index

tmt.index_relevance.delete_index(cr.content_directory)
tmt.index_relevance.calculate_relevance_index(cr.content_directory)

removed relevance index file:  data_sets/mixed/txt/index_relevance.hdf5
saving corpus relevance index ...  data_sets/mixed/txt/index_relevance.hdf5


In [8]:
# calculate document similarities

# delete if already exists
tmt.index_doc_similarity.delete_matrix(cr.content_directory)

# calclate doc1-doc2 similarity
tmt.index_doc_similarity.create_doc_similarity_matrix(cr.content_directory)

removed doc similarity matrix file:  data_sets/mixed/txt/matrix.docsimilarity
created  data_sets/mixed/txt/matrix.docsimilarity


In [9]:
#tmt.index_doc_similarity.print_matrix(cr.content_directory)

In [10]:
# get document similarities order by value
doc1_doc2_similarity = tmt.index_doc_similarity.get_doc_pairs_by_similarity(cr.content_directory)
doc1_doc2_similarity[:20]

,doc1,doc2,similarity
0,C05739561.txt,C05739546.txt,1.000000
1,C05739561.txt,C05739554.txt,0.997375
2,C05739565.txt,C05739563.txt,0.992705
3,C05739554.txt,C05739546.txt,0.991492
4,the-report-of-the-iraq-inquiry_section-33.txt,the-report-of-the-iraq-inquiry_section-32.txt,0.950962
5,the-report-of-the-iraq-inquiry_section-34.txt,the-report-of-the-iraq-inquiry_section-33.txt,0.926943
6,the-report-of-the-iraq-inquiry_section-34.txt,the-report-of-the-iraq-inquiry_section-32.txt,0.925336
7,the-report-of-the-iraq-inquiry_section-32.txt,the-report-of-the-iraq-inquiry_section-31.txt,0.916514
8,the-report-of-the-iraq-inquiry_section-34.txt,the-report-of-the-iraq-inquiry_section-31.txt,0.887027
9,the-report-of-the-iraq-inquiry_section-33.txt,the-report-of-the-iraq-inquiry_section-31.txt,0.858477


In [20]:
# visualise the doc similarities
tmt.visualisation.plot_force_directed_graph(doc1_doc2_similarity[:200])

force-directed graph